In [262]:
import numpy as np 
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import re
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [263]:
movie_data = pd.read_csv("set_movie.csv")
movie_df = pd.DataFrame(movie_data)
movie_df.head()

,Unnamed: 0,budget,genres,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,keywords,cast,crew,year,WR
0,1,4000000,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",13310,sv,Låt den rätte komma in,"Set in 1982 in the suburb of Blackeberg, Stock...",7.274237,2008-01-26,10785801.0,115.0,Released,Let the Right One In,7.5,997.0,"['female nudity', 'vampire', 'castration', 'bu...","[{'cast_id': 1, 'character': 'Oskar', 'credit_...","[{'credit_id': '52fe455a9251416c75053821', 'de...",2008.0,6.498874
1,8,35000000,"[{'id': 18, 'name': 'Drama'}]",9701,en,North Country,A fictionalized account of the first major suc...,8.781298,2005-02-12,25224242.0,126.0,Released,North Country,6.9,127.0,"['rape', 'secret', 'minnesota', 'witness', 'mi...","[{'cast_id': 1, 'character': 'Josey Aimes', 'c...","[{'credit_id': '52fe451ec3a36847f80bd87b', 'de...",2005.0,6.263596
2,37,2900000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",123109,en,No One Lives,A gang of ruthless highway killers kidnap a we...,7.202832,2013-03-09,74918.0,86.0,Released,No One Lives,5.5,173.0,"['suicide', 'male nudity', 'female nudity', 's...","[{'cast_id': 3, 'character': 'Driver', 'credit...","[{'credit_id': '52fe4a8dc3a368484e159f4b', 'de...",2013.0,6.211304
3,74,10500000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",20002,hi,Chandni Chowk To China,"Based in Delhi's Chandni Chowk, orphaned Sidhu...",1.970481,2009-01-16,9400000.0,149.0,Released,Chandni Chowk To China,5.3,13.0,"['musical', 'bollywood']","[{'cast_id': 10, 'character': 'Sidhu', 'credit...","[{'credit_id': '52fe43d2c3a368484e00004d', 'de...",2009.0,6.239733
4,106,20000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",15092,en,Crank: High Voltage,Chelios faces a Chinese mobster who has stolen...,10.044642,2009-04-16,34560577.0,96.0,Released,Crank: High Voltage,5.9,878.0,"['female nudity', 'prostitute', 'horse race', ...","[{'cast_id': 1, 'character': 'Chev Chelios', '...","[{'credit_id': '52fe46389251416c7507138f', 'de...",2009.0,6.179846


In [264]:
Counter(movie_df.status)

Counter({'Released': 3153, 'Post Production': 1})

In [266]:
### status 1개 빼고 모두 Realeased라 제거한다.

In [265]:
movie_df.drop(columns=["status"],inplace=True)
movie_df.head()

,Unnamed: 0,budget,genres,id,original_language,original_title,overview,popularity,release_date,revenue,runtime,title,vote_average,vote_count,keywords,cast,crew,year,WR
0,1,4000000,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",13310,sv,Låt den rätte komma in,"Set in 1982 in the suburb of Blackeberg, Stock...",7.274237,2008-01-26,10785801.0,115.0,Let the Right One In,7.5,997.0,"['female nudity', 'vampire', 'castration', 'bu...","[{'cast_id': 1, 'character': 'Oskar', 'credit_...","[{'credit_id': '52fe455a9251416c75053821', 'de...",2008.0,6.498874
1,8,35000000,"[{'id': 18, 'name': 'Drama'}]",9701,en,North Country,A fictionalized account of the first major suc...,8.781298,2005-02-12,25224242.0,126.0,North Country,6.9,127.0,"['rape', 'secret', 'minnesota', 'witness', 'mi...","[{'cast_id': 1, 'character': 'Josey Aimes', 'c...","[{'credit_id': '52fe451ec3a36847f80bd87b', 'de...",2005.0,6.263596
2,37,2900000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",123109,en,No One Lives,A gang of ruthless highway killers kidnap a we...,7.202832,2013-03-09,74918.0,86.0,No One Lives,5.5,173.0,"['suicide', 'male nudity', 'female nudity', 's...","[{'cast_id': 3, 'character': 'Driver', 'credit...","[{'credit_id': '52fe4a8dc3a368484e159f4b', 'de...",2013.0,6.211304
3,74,10500000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",20002,hi,Chandni Chowk To China,"Based in Delhi's Chandni Chowk, orphaned Sidhu...",1.970481,2009-01-16,9400000.0,149.0,Chandni Chowk To China,5.3,13.0,"['musical', 'bollywood']","[{'cast_id': 10, 'character': 'Sidhu', 'credit...","[{'credit_id': '52fe43d2c3a368484e00004d', 'de...",2009.0,6.239733
4,106,20000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",15092,en,Crank: High Voltage,Chelios faces a Chinese mobster who has stolen...,10.044642,2009-04-16,34560577.0,96.0,Crank: High Voltage,5.9,878.0,"['female nudity', 'prostitute', 'horse race', ...","[{'cast_id': 1, 'character': 'Chev Chelios', '...","[{'credit_id': '52fe46389251416c7507138f', 'de...",2009.0,6.179846


In [241]:
movie_df = movie_df.astype({"budget":"int"})
movie_df["release_date"] = pd.to_datetime(movie_df["release_date"])
movie_df = movie_df.astype({"year":"int"})

In [242]:
# 영향을 주지 않을
movie_df = movie_df.drop([movie_df.index[1188],movie_df.index[2353],movie_df.index[2175],movie_df.index[2398],movie_df.index[3038]])
movie_df.drop(columns=["original_title",'Unnamed: 0',"vote_count","vote_average","id","overview","title","keywords","crew","release_date","year","original_language"],inplace=True)
movie_df.head()

,budget,genres,popularity,revenue,runtime,cast,WR
0,4000000,"[{'id': 27, 'name': 'Horror'}, {'id': 18, 'nam...",7.274237,10785801.0,115.0,"[{'cast_id': 1, 'character': 'Oskar', 'credit_...",6.498874
1,35000000,"[{'id': 18, 'name': 'Drama'}]",8.781298,25224242.0,126.0,"[{'cast_id': 1, 'character': 'Josey Aimes', 'c...",6.263596
2,2900000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",7.202832,74918.0,86.0,"[{'cast_id': 3, 'character': 'Driver', 'credit...",6.211304
3,10500000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1.970481,9400000.0,149.0,"[{'cast_id': 10, 'character': 'Sidhu', 'credit...",6.239733
4,20000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",10.044642,34560577.0,96.0,"[{'cast_id': 1, 'character': 'Chev Chelios', '...",6.179846


In [243]:
# 대표 배우 1명
movie_df["cast"] = movie_df["cast"].apply(literal_eval)
movie_df["cast_cnt"] = movie_df["cast"].apply(lambda x: len(x)) 
movie_df["cast"] = movie_df["cast"].apply(lambda x : [i["name"] for i in x] if isinstance(x, list) else [])
movie_df["cast"] = movie_df["cast"].apply(lambda x : x[:1])

In [244]:
for i in range(len(movie_df["cast"])):
    try:
         movie_df["cast"][i] = [re.sub(r'[^\w]', '_',re.sub(r'[^\w\s]', '', entry)) for entry in movie_df["cast"][i]]
    except:
        movie_df["cast"][i]  = ['']

C:\Users\alal8\AppData\Local\Temp/ipykernel_18728/3272636718.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df["cast"][i] = [re.sub(r'[^\w]', '_',re.sub(r'[^\w\s]', '', entry)) for entry in movie_df["cast"][i]]
C:\Users\alal8\AppData\Local\Temp/ipykernel_18728/3272636718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df["cast"][i]  = ['']


In [245]:
# 많이 등장하는 배우 이름 top20
actor_count = Counter([name for names_per_file in movie_df.cast.values for name in names_per_file])
most_poppular_actors = [entry[0] for entry in actor_count.most_common(20)]
most_poppular_actors

['Nicolas_Cage',
 'Matt_Damon',
 'Johnny_Depp',
 'Bruce_Willis',
 'Denzel_Washington',
 'Adam_Sandler',
 'Mark_Wahlberg',
 'Ben_Stiller',
 'Ben_Affleck',
 'George_Clooney',
 'Tom_Cruise',
 'Christian_Bale',
 'Dwayne_Johnson',
 'Jason_Statham',
 'Tom_Hanks',
 'Vin_Diesel',
 'Keanu_Reeves',
 'Colin_Farrell',
 'Robert_De_Niro',
 'Owen_Wilson']

In [246]:
movie_df["genres"] = movie_df["genres"].fillna("[]").apply(literal_eval).apply(lambda x : [i["name"] for i in x] if isinstance(x, list) else [])
movie_df["genres"] = movie_df["genres"].apply(lambda x : x[:1])

In [247]:
movie_df.head()

,budget,genres,popularity,revenue,runtime,cast,WR,cast_cnt
0,4000000,[Horror],7.274237,10785801.0,115.0,[Kåre_Hedebrant],6.498874,15
1,35000000,[Drama],8.781298,25224242.0,126.0,[Charlize_Theron],6.263596,17
2,2900000,[Horror],7.202832,74918.0,86.0,[Luke_Evans],6.211304,14
3,10500000,[Action],1.970481,9400000.0,149.0,[Akshay_Kumar],6.239733,9
4,20000000,[Action],10.044642,34560577.0,96.0,[Jason_Statham],6.179846,96


In [248]:
movie_df = movie_df.astype({"genres":"str"})
movie_df = movie_df.astype({"cast":"str"})

In [249]:
movie_dum= pd.get_dummies(movie_df)

In [250]:
movie_dum

,budget,popularity,revenue,runtime,WR,cast_cnt,genres_['Action'],genres_['Adventure'],genres_['Animation'],genres_['Comedy'],...,cast_['Zhao_Wei'],cast_['Zoe_Saldana'],cast_['Zoe_Sloane'],cast_['Zoe_Weizenbaum'],cast_['Zoey_Deutch'],cast_['Zooey_Deschanel'],cast_['Zsófia_Psotta'],cast_['Гордан_Кичић'],cast_['杰森莫玛'],cast_[]
0,4000000,7.274237,1.078580e+07,115.0,6.498874,15,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35000000,8.781298,2.522424e+07,126.0,6.263596,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2900000,7.202832,7.491800e+04,86.0,6.211304,14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10500000,1.970481,9.400000e+06,149.0,6.239733,9,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20000000,10.044642,3.456058e+07,96.0,6.179846,96,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3149,200000000,14.477677,1.028571e+09,97.0,6.536127,31,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3150,20000000,8.468401,5.503005e+07,112.0,6.241065,19,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3151,20000000,4.850968,2.128186e+06,85.0,6.185706,20,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3152,110000,1.104156,2.700000e+05,165.0,6.243876,26,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [259]:
X = movie_dum.drop(columns=["WR"]).values
y = movie_dum["WR"].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=13)

In [260]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [261]:
pred_tr = reg.predict(X_train)
pred_test = reg.predict(X_test)
rmse_tr = (np.sqrt(mean_squared_error(y_train, pred_tr)))
rmse_test = (np.sqrt(mean_squared_error(y_test, pred_test)))

print("RMSE of Train Data : ", rmse_tr)
print("RMSE of Test Data : ", rmse_test)

RMSE of Train Data :  0.11985194936473062
RMSE of Test Data :  0.16994250222452548


In [267]:
X = movie_dum.drop(columns=["WR","cast_cnt"]).values
y = movie_dum["WR"].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=13)

In [268]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [269]:
pred_tr = reg.predict(X_train)
pred_test = reg.predict(X_test)
rmse_tr = (np.sqrt(mean_squared_error(y_train, pred_tr)))
rmse_test = (np.sqrt(mean_squared_error(y_test, pred_test)))

print("RMSE of Train Data : ", rmse_tr)
print("RMSE of Test Data : ", rmse_test)

RMSE of Train Data :  0.12054343335948868
RMSE of Test Data :  0.1719002629849056


In [270]:
X = movie_dum.drop(columns=["WR","runtime"]).values
y = movie_dum["WR"].values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.3, random_state=13)

In [271]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [272]:
pred_tr = reg.predict(X_train)
pred_test = reg.predict(X_test)
rmse_tr = (np.sqrt(mean_squared_error(y_train, pred_tr)))
rmse_test = (np.sqrt(mean_squared_error(y_test, pred_test)))

print("RMSE of Train Data : ", rmse_tr)
print("RMSE of Test Data : ", rmse_test)

RMSE of Train Data :  0.12364941918883975
RMSE of Test Data :  0.16966494099191134
